In [14]:
# Predicting the Disease from Symptom entered by User
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder

mlb = MultiLabelBinarizer()
le_disease = LabelEncoder()
le_prescription = LabelEncoder()

df['SYMPTOMS'] = df['SYMPTOMS'].apply(lambda x: ', '.join(str(x).split(', ')))

df['SYMPTOMS'] = df['SYMPTOMS'].str.lower()

df['SYMPTOMS'] = df['SYMPTOMS'].apply(lambda x: set(x.split(', ')))
symptoms_encoded = mlb.fit_transform(df['SYMPTOMS'])
df_encoded = pd.concat([df, pd.DataFrame(symptoms_encoded, columns=mlb.classes_)], axis=1)

X = df_encoded[mlb.classes_]
y_disease = le_disease.fit_transform(df['DISEASE'])
y_prescription = le_prescription.fit_transform(df['PRESCRIPTION'])

rf_disease_model = RandomForestClassifier(random_state=42)
rf_disease_model.fit(X, y_disease)

rf_prescription_model = RandomForestClassifier(random_state=42)
rf_prescription_model.fit(X, y_prescription)

def predict_disease_and_prescription(user_input, model_disease, model_prescription, symptom_encoder, disease_encoder, prescription_encoder):

    input_symptoms = set(str(user_input).lower().split(', '))
    unknown_labels = input_symptoms - set(symptom_encoder.classes_)

    if unknown_labels:
        input_symptoms -= unknown_labels
        print(f"Unknown labels removed from user input: {unknown_labels}")

    user_input_df = pd.DataFrame(columns=mlb.classes_)
    user_input_df.loc[0, :] = 0
    user_input_df.loc[0, input_symptoms] = 1
    predicted_disease = disease_encoder.inverse_transform([model_disease.predict(user_input_df)])[0]
    predicted_prescription = prescription_encoder.inverse_transform([model_prescription.predict(user_input_df)])[0]
    return predicted_disease, predicted_prescription

user_symptoms = input("Enter symptoms (comma separated): ")    # Enter Symptom
predicted_disease, predicted_prescription = predict_disease_and_prescription(user_symptoms, rf_disease_model, rf_prescription_model, mlb, le_disease, le_prescription)
print(f'Predicted Disease: {predicted_disease}')
print(f'Predicted Prescription: {predicted_prescription}','\n')

Enter symptoms (comma separated): Typically no symptoms until a fracture occurs
Predicted Disease: Osteoporosis 
Predicted Prescription: Alendronate, Risedronate, Calcium and Vitamin D supplements 



<ipython-input-14-ae9fe0a6f184>:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  user_input_df.loc[0, input_symptoms] = 1
<ipython-input-14-ae9fe0a6f184>:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  user_input_df.loc[0, input_symptoms] = 1
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
